# Dog Breed Identification System Using Deep Learning 🐕

## Introduction

In this project we shall use Deep Learning to classify images according to dog breed. This is a multi-class classification problem with 120 classes. Each class has a limited number of images. We are provided with a training and test set of images of dogs. Each image has a filename that is its unique id. The dataset comprises 120 breeds of dogs. The goal of the project is to create a classifier capable of determining a dog's breed from a photo.

### Dataset

The dataset for this project is available on Kaggle. <br>

**Link** : https://www.kaggle.com/c/dog-breed-identification/data

### Evaluation

We shall use <code>Accuracy</code>, <code>Precision</code>, <code>Recall</code> and <code>F1 score</code> to evaluate the performance of our models.<br>

Kaggle submissions are evaluated on <code>Multi Class Log Loss</code> between the predicted probability and the observed target.

## Table of Contents

1. Environment Setup
2. Dataset Gathering
3. Exploratory Data Analysis
4. Dataset Preprocessing
5. Model Experimentation
6. Model Evaluation

## Environment Setup

In [ ]:
# Suppressing Jupyter Notebook Warnings
import warnings
warnings.filterwarnings("ignore")

import os
import random

import tqdm

# Data manipulation libraries
import numpy as np
import pandas as pd

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# scikit-learn packages
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

# Deep Learning libraries
import tensorflow as tf
import tensorflow_hub as hub

import cv2

# Image display
from IPython.display import display, Image

## Dataset Gathering

In [ ]:
# Importing the labels dataset
labels_csv = pd.read_csv('../input/dog-breed-identification/labels.csv')

# Viewing the head of the dataset
labels_csv.head()

In [ ]:
# Saving the training dataset path to a variable
train_path = "../input/dog-breed-identification/train/"

# Creating image paths from the name
filenames = [train_path + fname + ".jpg" for fname in labels_csv['id']]

# Viewing the first 10 filenames
filenames[:10]

In [ ]:
# Checking whether the number of filenames in the directory matches to that of ours
if len(os.listdir(train_path)) == len(filenames):
    print('Matched !')
else:
    print('Not matched !')

## Exploratory Data Analysis

In [ ]:
# Viewing an image using filename
Image("../input/dog-breed-identification/train/000bec180eb18c7604dcecc8fe0dba07.jpg")

In [ ]:
# Viewing an image using our filenames variable
Image(filenames[10])

In [ ]:
# Visualizing the distribution of images accoding to class
labels_csv["breed"].value_counts().plot.bar(figsize=(20, 10));

## Dataset Preprocessing

In [ ]:
# Converting the label columns to Numpy array
labels = labels_csv['breed'].to_numpy()

# Viewing the first 10 labels
labels[:10]

In [ ]:
# Saving the count of total number of unique breeds to a variabkle
unique_breeds = np.unique(labels)

print("Total number of unique breeds : ", len(unique_breeds))

In [ ]:
# Converting the labels to a boolean array
boolean_labels = [label == np.array(unique_breeds) for label in labels]

# Viewing how it looks like
boolean_labels[0]

In [ ]:
# Creating training and validation sets

# Separating the features and labels
X = filenames
y = boolean_labels

print(f"Number of training images: {len(X)}")
print(f"Number of labels: {len(y)}")

X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y, 
                                                  test_size=0.2,
                                                  random_state=42)

print(f"Number of training images : {len(X_train)}")
print(f"Number of validation images : {len(X_val)}")

### Image Preprocessing

In [ ]:
# Reading an image in and checking shape
image = plt.imread(filenames[42])
print(f"Image Shape : {image.shape}")

# Converting the image to a Tensorflow Tensor
tf.constant(image)

In [ ]:
# Setting the Image Size
IMAGE_SIZE = 224

# Creating a function to preprocess the images
def process_image(image_path):
    '''
    This function shall preprocess the image
    1. Read in the image file
    2. Turn the image into numerical tensor
    3. Convert the color channel values to 0-1
    4. Resize the image
    '''
    
    # 1. Read in the image
    image = tf.io.read_file(image_path)
    
    # 2. Turn the image into numerical tensors
    image = tf.image.decode_jpeg(image, channels=3)
    
    # 3. Convert the color channel values from 0-225 to 0-1
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # 4. Resize the image
    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])
    
    return image

### Batching The Data

In [ ]:
# Creating a function to return a tuple (image, label)
def get_image_label(image_path, label):
    """
    Takes an image file path name and the associated label,
    processes the image and returns a tuple of (image, label).
    """
    image = process_image(image_path)
    return image, label

In [ ]:
# Setting the batch size at 32 
BATCH_SIZE = 32

# Create a function to turn data into batches
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
    """
    Function to batch the data
    """
    # If the data is a test dataset, we probably don't have labels
    if test_data:
        print("Creating test data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only filepaths
        data_batch = data.map(process_image).batch(BATCH_SIZE)
        return data_batch
  
    # If the data if a valid dataset, we don't need to shuffle it
    elif valid_data:
        print("Creating validation data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                                   tf.constant(y))) # labels
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
        return data_batch

    else:
        # If the data is a training dataset, we shuffle it
        print("Creating training data batches...")
        # Turn filepaths and labels into Tensors
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                                   tf.constant(y))) # labels
    
        # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
        data = data.shuffle(buffer_size=len(x))

        # Create (image, label) tuples (this also turns the image path into a preprocessed image)
        data = data.map(get_image_label)

        # Turn the data into batches
        data_batch = data.batch(BATCH_SIZE)
    return data_batch

In [ ]:
# Creating training and validation data batches
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [ ]:
# Checking the different attributes of our data batches
train_data.element_spec, val_data.element_spec

## Model Experimentation

### MobileNetV2

In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [None, IMAGE_SIZE, IMAGE_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = len(unique_breeds)

# Model URL for MobileNetV2
MODEL_URL1 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5'

# Creating Tensorflow EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                  patience=5)

In [ ]:
# Creating the MobileResNetV2 model
model1 = tf.keras.Sequential([
    # Layer 1 : Input Layer
    hub.KerasLayer(MODEL_URL1),
    
    # Layer 2 : Output Layer
    tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                          activation='softmax')
])

# Compiling the model
model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

# Building the model
model1.build(INPUT_SHAPE)

# Summary of the model
model1.summary()

In [ ]:
# Fitting the model
history1 = model1.fit(train_data,
                      epochs=100,
                      validation_data=val_data,
                      callbacks=[early_stopping])

### EfficientNetV2

In [ ]:
# Model URL for EfficientNetV2
MODEL_URL2 = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/classification/2'

# Creating model for EfficientNetV2
model2 = tf.keras.Sequential([
    # Layer 1 : Input Layer
    hub.KerasLayer(MODEL_URL2),
    
    # Layer 2 : Output Layer
    tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                          activation='softmax')
])

# Compiling the model
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

# Building the model
model2.build(INPUT_SHAPE)

# Summary of the model
model2.summary()

In [ ]:
# Fitting the model
history2 = model2.fit(train_data,
                      epochs=100,
                      validation_data=val_data,
                      callbacks=[early_stopping])

### ResNet50V1

In [ ]:
# Model URL for ResNet50V2
MODEL_URL3 = "https://tfhub.dev/tensorflow/resnet_50/classification/1"

# Creating the model for ResNet50V2
model3 = tf.keras.Sequential([
    # Layer 1 : Input Layer
    hub.KerasLayer(MODEL_URL3),
    
    # Layer 2 : Output Layer
    tf.keras.layers.Dense(120, activation='softmax')
])

# Compiling the model
model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

# Building the model
model3.build(INPUT_SHAPE)

# Summary of the model
model3.summary()

In [ ]:
# Fitting the model
history3 = model3.fit(train_data,
                      epochs=100,
                      validation_data=val_data,
                      callbacks=[early_stopping])

### InceptionV3

In [ ]:
# Model URL for InceptionV3
MODEL_URL4 = 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5'

# Creating model for InceptionV3
model4 = tf.keras.Sequential([
    # Layer 1 : Input Layer
    hub.KerasLayer(MODEL_URL4),
    
    # Layer 2 : Output Layer
    tf.keras.layers.Dense(120, activation='softmax')
])

# Compiling the model
model4.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

# Building the model
model4.build(INPUT_SHAPE)

# Summary of the model
model4.summary()

In [ ]:
# Fitting the model
history4 = model4.fit(train_data,
                      epochs=100,
                      validation_data=val_data,
                      callbacks=[early_stopping])

## Model Evaluation

In [ ]:
# Creating graphs to visualize the accuracy and loss for the models
fig, axes = plt.subplots(nrows=4, 
                         ncols=2, 
                         figsize=(15, 25))

fig.tight_layout(pad=5)

plt.style.use('fivethirtyeight')

# - *********************** - #
# Graph for MobileNetV2 Training Accuracy vs Validation Accuracy
axes[0][0].plot(history1.history['accuracy'])
axes[0][0].plot(history1.history['val_accuracy'])
axes[0][0].set_ylabel("Accuracy")
axes[0][0].set_xlabel("Epochs")
axes[0][0].set_title('Model 1: MobileNetV2 Train Acc vs Val Acc')
axes[0][0].legend(['Train', 'Test'], loc='upper left')

# Graph for MobileNetV2 Training Loss vs Validation Loss
axes[0][1].plot(history1.history['loss'])
axes[0][1].plot(history1.history['val_loss'])
axes[0][1].set_ylabel("Loss")
axes[0][1].set_xlabel("Epochs")
axes[0][1].set_title('Model 1: MobileNetV2 Train Loss vs Val Loss')
axes[0][1].legend(['Train', 'Test'], loc='upper left')
# - *********************** - #

# - *********************** - #
# Graph for EfficientNetV2 Training Accuracy vs Validation Accuracy
axes[1][0].plot(history2.history['accuracy'])
axes[1][0].plot(history2.history['val_accuracy'])
axes[1][0].set_ylabel("Accuracy")
axes[1][0].set_xlabel("Epochs")
axes[1][0].set_title('Model 2: EfficientNet50V2 Train Acc vs Val Acc')
axes[1][0].legend(['Train', 'Test'], loc='upper left')

# Graph for EfficientNetV2 Training Loss vs Validation Loss
axes[1][1].plot(history2.history['loss'])
axes[1][1].plot(history2.history['val_loss'])
axes[1][1].set_ylabel("Loss")
axes[1][1].set_xlabel("Epochs")
axes[1][1].set_title('Model 2: EfficientNet50V2 Train Loss vs Val Loss')
axes[1][1].legend(['Train', 'Test'], loc='upper left')
# - *********************** - #

# - *********************** - #
# Graph for ResNet50V2 Training Accuracy vs Validation Accuracy
axes[2][0].plot(history3.history['accuracy'])
axes[2][0].plot(history3.history['val_accuracy'])
axes[2][0].set_ylabel("Accuracy")
axes[2][0].set_xlabel("Epochs")
axes[2][0].set_title('Model 3: ResNet50V2 Train Acc vs Val Acc')
axes[2][0].legend(['Train', 'Test'], loc='upper left')

# Graph for EfficientNetV2 Training Loss vs Validation Loss
axes[2][1].plot(history3.history['loss'])
axes[2][1].plot(history3.history['val_loss'])
axes[2][1].set_ylabel("Loss")
axes[2][1].set_xlabel("Epochs")
axes[2][1].set_title('Model 3: ResNet50V2 Train Loss vs Val Loss')
axes[2][1].legend(['Train', 'Test'], loc='upper left')
# - *********************** - #

# - *********************** - #
# Graph for InceptionV3 Training Accuracy vs Validation Accuracy
axes[3][0].plot(history4.history['accuracy'])
axes[3][0].plot(history4.history['val_accuracy'])
axes[3][0].set_ylabel("Accuracy")
axes[3][0].set_xlabel("Epochs")
axes[3][0].set_title('Model 4: InceptionV3 Train Acc vs Val Acc')
axes[3][0].legend(['Train', 'Test'], loc='upper left')

# Graph for InceptionV3 Training Loss vs Validation Loss
axes[3][1].plot(history3.history['loss'])
axes[3][1].plot(history3.history['val_loss'])
axes[3][1].set_ylabel("Loss")
axes[3][1].set_xlabel("Epochs")
axes[3][1].set_title('Model 4: InceptionV3 Train Loss vs Val Loss')
axes[3][1].legend(['Train', 'Test'], loc='upper left')
# - *********************** - #

### Champion Model

In [ ]:
# ResNet50V2 turns out to be the champion model
final_model = tf.keras.Sequential([
    # Layer 1 : Input Layer
    hub.KerasLayer(MODEL_URL3),
    
    # Layer 2 : Output Layer
    tf.keras.layers.Dense(120, activation='softmax')
])

# Compiling the model
final_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    metrics=['accuracy'],
                    optimizer=tf.keras.optimizers.Adam())

# Building the model
final_model.build(INPUT_SHAPE)

# Model Summary
final_model.summary()

In [ ]:
# Fitting the model
final_history = final_model.fit(train_data,
                                epochs=38)

In [ ]:
# Making predictions
predictions = final_model.predict(val_data,
                                  verbose=2)

# Viewing the predictions
predictions[0]

In [ ]:
# Checking the shape of the prediction
print("Viewing the Shape : ", predictions.shape)

# Checking the maximum probability
print(f"Maximum value (probability of prediction) : {np.max(predictions[0])}")

# Maximum index
print(f"Maximum index : {np.argmax(predictions[0])}")

# Predicted label
print(f"Predicted Label : {unique_breeds[np.argmax(predictions[0])]}")

In [ ]:
# Creating a function to unbatch the data
def unbatching(data):
    '''
    This fuction is used to unbatch the data
    '''
    # Creating variables to save the images and labels
    images = []
    labels = []
    
    # Looping through the unbatched data
    for image, label in data.unbatch().as_numpy_iterator():
        images.append(image)
        labels.append(unique_breeds[np.argmax(label)])
    return images, labels

# Unbatching the validation data
val_images, val_labels = unbatching(val_data)
val_images[0], val_labels[0]

In [ ]:
# Getting the predicted labels
predicted_labels = [unique_breeds[np.argmax(predictions[i])] for i in range(len(predictions))]

# Checking to see if the length of the predicted labels is equal to the toal number of data points in the validation dataset
if len(predicted_labels) == len(val_labels):
    print('Matched !')
else:
    print('Not Matched !')

In [ ]:
print(classification_report(val_labels, predicted_labels))

In [ ]:
# Getting the perfromance metrics of our champion model
print("#******** ResNet50V2 Performance Metrics ********#")
print(" ")
print(f"Accuracy Score  = {accuracy_score(val_labels, predicted_labels) * 100}")
print(f"Precision Score = {precision_score(val_labels, predicted_labels, average='macro') * 100}")
print(f"Recall Score    = {recall_score(val_labels, predicted_labels, average='macro') * 100}")
print(f"F1 Score        = {f1_score(val_labels, predicted_labels, average='macro') * 100}")